In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
from seaborn import kdeplot
import numpy as np
import zipfile

In [54]:
import scipy.stats as sp
import patsy as pt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import KernelDensity

In [55]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
pd.set_option('display.max_columns', None)

In [56]:
entrada = {'zipf': ['PNADC_012022_20220916', 'PNADC_022022_20220916','Dados_20170517', 
                    'Dicionario_e_input_20220224', 'Dicionarios_e_input_20170517'], 
          'subd': ['', '/Dicionários e input', '/Dados'], #SUBDIRETORIO
          'arq': ['dicionario_PNADC_microdados_trimestral', #Dicionarios
            'Dicionário de variáveis de pessoas - PNAD 2015',
            'Dicionário de variáveis de domicílios - PNAD 2015',
            'PNADC_012022', 'PNADC_022022', 'DOM2015', 'PES2015']}

#### Carregamento de dados no formato .fwf
[0][0:]pesquisas [1][0:] dicionarios || [0][0:]subdiretórios [1][0:]dicionarios [2][0:]microdados

In [57]:
def loadData(dicio, dados, n_fields=220, lastPos=3478):
    with zipfile.ZipFile(f'C:/DATASETS/IBGE/PNAD/{dicio[0]}.zip') as z:
        df_dict = pd.read_excel(z.open(f'{dicio[1]}{dicio[2]}.xls'), header=None, 
                              skiprows=4, dtype='string').dropna(how='all')[[0, 1, 2, 4, 5, 6]]

        sizes = df_dict[1][df_dict[1].str.isnumeric()].astype(int).values
        positions = df_dict[0][df_dict[0].str.isnumeric()].astype(int).values-1
        final_pos = [i[0]+i[1] for i in zip(positions, sizes)]
        final_pos[n_fields], final_pos = lastPos, final_pos[0:n_fields]
        tuples = list(zip(positions, final_pos))
    #dicionário de variáveis e tamanhos das colunas

    with zipfile.ZipFile(f'C:/DATASETS/IBGE/PNAD/{dados[0]}.zip') as z:
        pnad = pd.read_fwf(z.open(f'{dados[1]}{dados[2]}.txt'), 
                              colspecs=tuples, header=None, nrows=500000, dtype=str)

    df_dict.fillna(value=None, method='ffill', inplace=True)
    df_dict.set_index(2, inplace=True)
    df_dict.index = df_dict.index.str.lower()
    pnad.columns = list(df_dict.index.str.lower().dropna().unique()[0:n_fields]) #dataset carregado
    
    return pnad

In [58]:
pnad = loadData(dicio=[entrada['zipf'][3], entrada['subd'][0], entrada['arq'][0]], 
                dados=[entrada['zipf'][0], entrada['subd'][0], entrada['arq'][3]],
                n_fields=220, lastPos=3478)

KeyboardInterrupt: 

In [ ]:
pnad2 = loadData(dicio=[entrada['zipf'][3], entrada['subd'][0], entrada['arq'][0]], 
                dados=[entrada['zipf'][1], entrada['subd'][0], entrada['arq'][4]],
                n_fields=220, lastPos=3478)

In [ ]:
pnad = pd.concat([pnad, pnad2])
del pnad2

In [ ]:
def cat_dict(df, var): #substituir códigos pelo nome da categoria
    catDict = dict()
    for n, cat in enumerate(df_dict.loc[var][5]):
        catDict[cat] = df_dict.loc[var][6][n]
    return {var.lower(): catDict}

In [ ]:
def var_dict(df): #substituir nomes das colunas pela descrição, se tiver menos de 25 caracteres
    colDict = dict()
    for n, col in enumerate(df[4].drop_duplicates().index.unique()):
        if len(df[4].drop_duplicates()[n])<25:
            colDict[col] = df[4].drop_duplicates()[n]
    return colDict

In [ ]:
 def drop_min_obs(df, min_obs=10000): #excluir colunas que não tenha o min desejado de obs
    to_drop = list()
    for i in df.columns:
        n_obs = (df[i].isna()==False).sum()
        if n_obs<min_obs:
            to_drop.append(i)
        else:
            pass

    return df.drop(to_drop, axis=1, inplace=True)

#drop_min_obs(pnad, min_obs=50000)

In [ ]:
pnad.replace(cat_dict(df_dict, 'v2007')).rename(columns=var_dict(df_dict)).head()

In [ ]:
index=iter(df_dict.index.unique()) #explorar dicionário de categorias

In [ ]:
df_dict.loc[next(index)]

In [ ]:
pnad.shape

#### Explorar dados por índice nominal

In [ ]:
cap = iter(pnad['capital'].dropna().unique())

In [ ]:
# explorar dataframe por índices nominais
pnad.set_index('capital').loc[next(cap)].head()

In [ ]:
filtro_renda = ((pnad['v2005']=='03')|(pnad['v2005']=='02')|(pnad['v2005']=='01'))\
                  &(pnad['v2009'].astype(int)>=20)&(pnad['uf']=='31')
                    #&(pnad['v403312'].dropna().astype(int)>500)&(pnad['v403312'].dropna().astype(int)<=5000)

In [ ]:
ax, fig = plt.subplots(figsize=(12,6), facecolor='#084d6e')

for cor in np.delete(pnad['v2010'].dropna().unique(), -1):
    fig.hist(pnad[(pnad['v2009'].astype(int)>21)&(pnad['v2010']==cor)]['vd3005'].dropna().astype(int), bins=7, 
             alpha=1/len(np.delete(pnad['v2010'].dropna().unique(), -1)), color='orange')

ax2 = ax.gca().twinx()
for cor in np.delete(pnad['v2010'].dropna().unique(), -1):
    kdeplot(pnad[(pnad['v2009'].astype(int)>21)&(pnad['v2010']==cor)]['vd3005'].dropna().astype(int), 
            lw=1, label=cor, ax=ax2)
fig.set_facecolor('#084d6e')
plt.legend()
plt.title('Distribuição de frequência de anos estudados')
plt.show()

Teste K-S de normalidade da variável renda

In [ ]:
sp.stats.kstest(pnad['v403312'].dropna().astype(int), 'norm')

### Data Wranggling
• Determinação de domicílios com pais e filhos morando para comparar anos de estudo entre gerações<p>
    >> Se domicílio possui V2005=='04' (filho), basta verificar o identificador do domicílio no dataset completo para filtrar

In [ ]:
filtro_f = (pnad['v2005']=='04') & (pnad['v2009'].astype(int)>=25) #datasets de filhos e seus responsáveis
filtro_r = (pnad['v2005']=='01') & (pnad['v2009'].astype(int)>=25)

df_filhos = pnad[filtro_f][['uf', 'upa', 'estrato', 'v1008', 'vd3005', 'v2005', 'v2009']]
df_resp = pnad[filtro_r][['upa', 'estrato', 'v1008', 'vd3005', 'v2005', 'v2009']]

for i in [df_filhos, df_resp]: #unificando campos que identificam domicílios em uma primary key
    i['key'] = i['upa']+i['estrato']+i['v1008']
    i.drop(['upa','estrato','v1008'], axis=1, inplace=True)

df_fc = df_resp.merge(df_filhos, on='key', suffixes=['_r', '_f'])

In [ ]:
df_fc.head(3)

#### Comparando anos estudados pelos pais com os anos estudados dos filhos

In [ ]:
#tabela de contingência de anos estudados pelos responsáveis e pelos filhos
df_fc_resp = df_fc[['v2005_r', 'vd3005_r', 'key']].drop_duplicates() #não contar um pai mais de uma vez por ter mais de um filho
anos = pd.concat([df_fc_resp['vd3005_r'], df_fc['vd3005_f']]).astype(float)
cat = pd.concat([df_fc_resp['v2005_r'], df_fc['v2005_f']]).astype('category')

df_final = pd.concat([anos, cat], axis=1, ignore_index=True)
df_final.columns=['anos_estudados', 'relacao_familia']

df_final['anos_estudados_c'] = pd.cut(df_final['anos_estudados'].astype(int), bins=[0,4,7,11,15,16], 
                                      labels=['0 a 3', '4 a 7', '8 a 11', '12 a 15', '16+'])

In [ ]:
pd.crosstab(df_final['anos_estudados_c'],
            df_final['relacao_familia'].replace(cat_dict(df_dict, 'v2005')['v2005'])) #normalizada para checar porcentagens muito abaixo de 5%

In [ ]:
plt.hist(df_final[df_final['relacao_familia']=='01']['anos_estudados'].astype(int), alpha=0.5, color='blue', bins=5, align='left')
plt.hist(df_final[df_final['relacao_familia']=='04']['anos_estudados'].astype(int), alpha=0.5, color='orange', bins=5, align='left')
plt.title('Distribuição de frequência de anos estudados \npor responsáveis e seus filhos')
plt.show()

### Qui Quadrado
1. Discretização da variável "anos estudados" (vd3005)<p>
2. Tabela de contingência de faixa de anos de estudo por condição no domicílio <P>
3. Normalização por coluna. "Filho do responsável" de 0 a 3 anos deu pouco a baixo de 5%, mas como a amostragem é grande, realizei o teste assim mesmo.<p>
4. realizei um segundo teste com todos os responsáveis maiores de 35 anos (porque é média dos responsáveis que possuem filho no domicílio do df_final) e todos os filhos maiores de 22 anos.

In [ ]:
#qui quadrado com a biblioteca scipy
sp.chi2_contingency(pd.crosstab(df_final['anos_estudados_c'], df_final['relacao_familia']))

Teste incluindo responsáveis que não possuem filhos

In [ ]:
filtro_rf = ((pnad['v2005']=='01')&(pnad['v2009'].astype(int)>=36))|((pnad['v2009'].astype(int)>=22)&(pnad['v2005']=='04'))

pnad['vd3005_c']=pd.cut(pnad['vd3005'].dropna().astype(int), bins=[0,4,7,11,15,16], 
                                      labels=['0 a 3', '4 a 7', '8 a 11', '12 a 15', '16+'])

sp.chi2_contingency(pd.crosstab(pnad[filtro_rf]['vd3005_c'].astype('category'), pnad[filtro_rf]['v2005'].astype('category')))

### Teste T
• responsáveis acima de 35 anos (média dos responsáveis do dataset pareado)<p>
• filhos acima de 22 anos ()

In [ ]:
filtro_resp = (pnad['v2005']=='01')&(pnad['v2009'].astype(int)>=36)
filtro_filho = (pnad['v2005']=='04')&(pnad['v2009'].astype(int)>=22)

In [ ]:
ax, fig = plt.subplots() #plotar anos estudados sem que todos os responsáveis tenham um filho na amostra
fig.hist(pnad[filtro_resp]['vd3005'].astype(int), alpha=0.5, bins=6, color='blue')
ax2 = ax.gca().twinx() #dar escalas diferentes para cada grupo, pois n amostral é muito discrepante
ax2.hist(pnad[filtro_filho]['vd3005'].astype(int), alpha=0.5, bins=6, color='orange')

plt.show()

Sem filtragem de responsáveis -> n=133652 | filhos -> n=20506

In [ ]:
sp.ttest_ind(pnad[filtro_resp]['vd3005'].astype(int), pnad[filtro_filho]['vd3005'].astype(int), equal_var=True)

Com filtragem de responsáveis (todos têm filhos, alguns mais de um)<p>
    • responsáveis -> n=11857 <p>
    • filhos -> n=14415

In [ ]:
sp.ttest_ind(df_final[df_final['relacao_familia']=='01']['anos_estudados'],
             df_final[df_final['relacao_familia']=='04']['anos_estudados'])

In [ ]:
sp.stats.kstest(df_final['anos_estudados'], 'norm')

In [ ]:
def moradores_mesmo_domicilio(dfA, dfB):
    for i in [dfA, dfB]: #unificando campos que identificam domicílios em uma primary key
        i['key'] = i['upa']+i['estrato']+i['v1008']
        i.drop(['upa','estrato','v1008'], axis=1, inplace=True)

    dfa_dfb = dfA.merge(dfB, on='key', suffixes=['_r', '_f'])
    
    var_1 = list(a.keys())
    dfa_dfb_resp = dfa_dfb[[var_1[0]+'_r', var+'_r', 'key']].drop_duplicates() #não contar um pai mais de uma vez por ter mais de um filho
    anos = pd.concat([dfa_dfb_resp[var+'_r'], dfa_dfb[var+'_f']]).astype(float)
    cat = pd.concat([dfa_dfb_resp[var_1[0]+'_r'], dfa_dfb[var_1[0]+'_f']]).astype('category')

    dfa_dfb = pd.concat([anos, cat], axis=1, ignore_index=True)
    dfa_dfb.columns=['anos_estudados', 'relacao_familia']
    
    return dfa_dfb